In [1]:
!pip install opencv-contrib-python
!pip install requests

In [30]:
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as pyplot
import matplotlib.pyplot as plt

import uuid
import json
import time
import cv2
import requests
import os
import csv

In [31]:
api_url = ''
secret_key = ''

In [32]:
# CSV 파일에서 블러링을 적용할 글자 목록 읽기
def read_text_to_blur_from_csv(csv_file):
    text_to_blur = []
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if row:
                text_to_blur.append(row[0])
    return text_to_blur

In [33]:
# CSV 파일에서 블러링을 적용할 글자 목록 읽기
csv_file = 'Total_Road_Address.csv'
text_to_blur = read_text_to_blur_from_csv(csv_file)

In [34]:
#print(text_to_blur[1])

In [35]:
# 이미지를 처리하고 저장하는 함수 정의
def process_and_save_image(image_path, text_to_blur, output_folder):
    # 이미지 로드
    img = cv2.imread(image_path)
    roi_img = img.copy()
    original = img.copy()

    # CLOVA OCR API에 요청 보내기
    image_file = ('file', open(image_path, 'rb'))  # 이미지 파일을 튜플로 래핑

    request_json = {'images': [{'format': 'jpg',
                                'name': 'demo'
                               }],
                    'requestId': str(uuid.uuid4()),
                    'version': 'V2',
                    'timestamp': int(round(time.time() * 1000))
                   }

    payload = {'message': json.dumps(request_json).encode('UTF-8')}

    headers = {
      'X-OCR-SECRET': secret_key,
    }

    response = requests.request("POST", api_url, headers=headers, data=payload, files=[image_file]) # 이미지 파일을 리스트로 전달
    result = response.json()

    # OCR 결과 처리 및 블러링 적용
    for field in result['images'][0]['fields']:
      text = field['inferText']
      vertices_list = field['boundingPoly']['vertices']
      pts = [tuple(vertice.values()) for vertice in vertices_list]
      topLeft = [int(_) for _ in pts[0]]
      topRight = [int(_) for _ in pts[1]]
      bottomRight = [int(_) for _ in pts[2]]
      bottomLeft = [int(_) for _ in pts[3]]

      cv2.line(roi_img, topLeft, topRight, (0,255,0), 2)
      cv2.line(roi_img, topRight, bottomRight, (0,255,0), 2)
      cv2.line(roi_img, bottomRight, bottomLeft, (0,255,0), 2)
      cv2.line(roi_img, bottomLeft, topLeft, (0,255,0), 2)

      # 블러링 적용
      if text in text_to_blur:
         x1, y1 = topLeft
         x2, y2 = bottomRight
         if x1 < x2 and y1 < y2:  # 영역이 비어 있지 않을 때만 블러링 적용
            roi = img[y1:y2, x1:x2]
            if roi.size != 0: # 이미지 영역의 크기가 0이 아닌 경우에만 블러링 적용
              roi = cv2.GaussianBlur(roi, (0, 0), sigmaX=10)  # 블러링 효과 적용
              img[y1:y2, x1:x2] = roi
            else:
              print(f"Empty ROI at x1={x1}, y1={y1}, x2={x2}, y2={y2}")
         else:
          print(f"Invalid ROI coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

    # 이미지 저장 경로 설정
    output_path = os.path.join(output_folder, os.path.basename(image_path))

    # 이미지 저장
    cv2.imwrite(output_path, img)

In [36]:
# 이미지 파일이 있는 폴더 경로 설정
image_folder = 'OCR_Test'

# 이미지 파일 경로 목록 생성
image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith(('.jpg', '.png', '.jpeg'))]


In [37]:
#print(image_paths)

In [38]:
# 이미지를 처리하고 저장하는 루프
output_folder = 'output_folder_1'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for image_path in image_paths:
    process_and_save_image(image_path, text_to_blur, output_folder)

Invalid ROI coordinates: x1=191, y1=214, x2=573, y2=203
Invalid ROI coordinates: x1=322, y1=563, x2=668, y2=485
Invalid ROI coordinates: x1=371, y1=113, x2=516, y2=113
Empty ROI at x1=-9, y1=66, x2=324, y2=120
Empty ROI at x1=-1, y1=149, x2=188, y2=190
Invalid ROI coordinates: x1=282, y1=170, x2=471, y2=132
Invalid ROI coordinates: x1=418, y1=199, x2=578, y2=179
